In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/poetry/Kanye_West.txt
/kaggle/input/poetry/johnny-cash.txt
/kaggle/input/poetry/kanye-west.txt
/kaggle/input/poetry/bruno-mars.txt
/kaggle/input/poetry/dickinson.txt
/kaggle/input/poetry/amy-winehouse.txt
/kaggle/input/poetry/blink-182.txt
/kaggle/input/poetry/paul-simon.txt
/kaggle/input/poetry/patti-smith.txt
/kaggle/input/poetry/bieber.txt
/kaggle/input/poetry/disney.txt
/kaggle/input/poetry/jimi-hendrix.txt
/kaggle/input/poetry/lin-manuel-miranda.txt
/kaggle/input/poetry/adele.txt
/kaggle/input/poetry/dj-khaled.txt
/kaggle/input/poetry/beatles.txt
/kaggle/input/poetry/r-kelly.txt
/kaggle/input/poetry/lady-gaga.txt
/kaggle/input/poetry/radiohead.txt
/kaggle/input/poetry/britney-spears.txt
/kaggle/input/poetry/alicia-keys.txt
/kaggle/input/poetry/rihanna.txt
/kaggle/input/poetry/joni-mitchell.txt
/kaggle/input/poetry/dolly-parton.txt
/kaggle/input/poetry/drake.txt
/kaggle/input/poetry/Lil_Wayne.txt
/kaggle/input/poetry/notorious_big.txt
/kaggle/input/poetry/eminem.txt
/

In [2]:
import tensorflow as tf
import keras
import re
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, SimpleRNN, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences

# Define the functions to create a model based off an artists lyrics and generate new lyrics from that model

In [3]:
chars = list('abcdefghijklmnopqrstuvwxyz')
chars.append(' ')
chars.append('\n')
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_window = 20
step = 3

In [4]:
def create_model(artist):
    
    with open('/kaggle/input/poetry/'+artist+'.txt') as f:
        text = f.read().lower()

    text = re.sub('[^a-z \n]','',text)

    inputs = []
    next_chars = []

    for i in range(0,len(text)-char_window,step):
        inputs.append(text[i:i+char_window])
        next_chars.append(text[i+char_window])

    pairs = zip(inputs,next_chars)
    list(pairs)[:100]

    X = np.zeros((len(inputs),char_window),dtype=np.float32)

    for i in range(len(inputs)):
        for j in range(char_window):
            X[i,j] = char_to_index[inputs[i][j]]

    y = np.zeros(len(next_chars),dtype=np.float32)

    for i in range(len(next_chars)):
        y[i] = char_to_index[next_chars[i]]

    y = to_categorical(y)
    
    model = Sequential()

    model.add(Embedding(28,64,input_length=char_window))
    model.add(LSTM(256,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(28,activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    model.fit(X,y,epochs=5,batch_size=20)
    model.save('/kaggle/working/'+artist)
    return model

In [5]:
def softmax(z):
    return np.exp(z)/sum(np.exp(z))

def temp_scale(conditional_probability, temperature=1.0):
    conditional_probability = np.asarray(conditional_probability).astype("float64")
    conditional_probability = np.log(conditional_probability) / temperature
    reweighted_conditional_probability = softmax(conditional_probability)
    probas = np.random.multinomial(1, reweighted_conditional_probability, 1)
    return np.argmax(probas)

def generate_line(model,context,temp):
    line = context
    next_char = ''
    counter = 0
    while counter<100 and next_char != '\n':
        X = [[char_to_index[line[i]] for i in range(0,len(line))]]
        X = pad_sequences(X,maxlen=char_window)
        y = model.predict(X)
        next_char = index_to_char[temp_scale(y[0,:],temp)]
        counter+=1
        line+=next_char
    return line

# Now lets have some fun emulating different artists!!

In [6]:
kanye = create_model('Kanye_West')

2022-03-16 03:02:31.222753: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-03-16 03:02:31.943863: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
4183/4183 [==============================] - 319s 75ms/step - loss: 2.2064 - accuracy: 0.3462
Epoch 2/5
4183/4183 [==============================] - 319s 76ms/step - loss: 1.7782 - accuracy: 0.4507
Epoch 3/5
4183/4183 [==============================] - 321s 77ms/step - loss: 1.6431 - accuracy: 0.4867
Epoch 4/5
4183/4183 [==============================] - 322s 77ms/step - loss: 1.5498 - accuracy: 0.5120
Epoch 5/5
4183/4183 [==============================] - 319s 76ms/step - loss: 1.4787 - accuracy: 0.5321


2022-03-16 03:29:15.359322: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [7]:
temps = [.01, 0.2, 0.5, 1.0, 1.2, 2]

for temp in temps:
    print('\n\n')
    print(generate_line(kanye,'i had a dream i could',temp))




i had a dream i could be a bad the streets the same the same the same the same the same the same the same the same the sa



i had a dream i could be and a from a dreams the soul and i aint talk the same in the best the deal the chool the chain t



i had a dream i couldnt jesus in the heres to more that dont get the more before i got the last but i aint fain the arr t



i had a dream i could my girl white from callin




i had a dream i couldnt you know nover nighttinges kanye




i had a dream i couldmade hih nright defuliim



In [8]:
for temp in temps:
    print('\n\n')
    print(generate_line(kanye,'one good girl',temp))




one good girl in the chain the streets the same the same the same the same the same the same the same the same th



one good girl what i aint gotta got the chain shit




one good girl die the way the shit its all these dont started on the chain the mars more i live the game you say 



one good girl




one good girl had last on out




one good girl gake iajyss do bring mirwerinnnacyghligceaar yal



In [9]:
for temp in temps:
    print('\n\n')
    print(generate_line(kanye,'i need you',temp))




i need your grave the streets the streets the same the same the same the same the same the same the same the s



i need you got the same the best fresh on the stram that who know when i get the streets the best from the bab



i need you should i seen do the superfont that girl ass on my girls




i need youl




i need you rens by fast to so thee usingsod on my




i need you voincnig ty tdihrsael manaed



In [10]:
jcash = create_model('johnny-cash')

Epoch 1/5
1445/1445 [==============================] - 114s 76ms/step - loss: 2.5054 - accuracy: 0.2709
Epoch 2/5
1445/1445 [==============================] - 112s 78ms/step - loss: 2.0417 - accuracy: 0.3835
Epoch 3/5
1445/1445 [==============================] - 112s 78ms/step - loss: 1.8257 - accuracy: 0.4387
Epoch 4/5
1445/1445 [==============================] - 112s 77ms/step - loss: 1.6539 - accuracy: 0.4885
Epoch 5/5
1445/1445 [==============================] - 109s 76ms/step - loss: 1.5004 - accuracy: 0.5308


In [11]:
for temp in temps:
    print(generate_line(jcash,'im going to',temp))

im going to show i was a hand the rong of the road and i was a hand the rong of the road and i want to the rong
im going to show to the rond of my baby me when the sont and well be when the reaser in the seen when the town 
im going to ray

im going to good caumer around were eved jacksiness this come of rust threw the omen the miolseer try mut the f
im going to douth and huve a locd all lishy memody and well co a but you keep hud you mas lair god for uw bloke
im going to corracb



In [12]:
for temp in temps:
    print(generate_line(jcash,'and it burns burns burns',temp))

and it burns burns burns me and i want to the rong of the road and i want to the rong of the road and i want to the rong of 
and it burns burns burns and someone when the sont be when the sould of fir the ring of the road again the blues

and it burns burns burns me and ill been my befp me when the sweethine that was a little steeldray

and it burns burns burns boolly down

and it burns burns burns telld for a alonns of ank the teclssed steetstoothy shot

and it burns burns burns



In [13]:
for temp in temps:
    print(generate_line(jcash,'i shot a man',temp))

i shot a man in the seen when the sont and well the rong of the road and i was a hand the rong of the road and i
i shot a man

i shot a man then you to carry me home me me when the deves to see the tolght of the rood well the like to so sh
i shot a man

i shot a many carm

i shot a mandangacapacry

